In [1]:
import pandas
import networkx
import os
import json
import pandas as pd

In [2]:
#first, get the landpoint coordinates:
landPointDf = pd.read_csv("landpoints.csv")

cityLookup = {}
for l in landPointDf.itertuples():
    #add the city name for all 4 possible coordinates
    longitude = int(l.longitude)
    latitude = int(l.latitude)
    cityLookup[(longitude, latitude)] = l.description
    cityLookup[(-longitude, latitude)] = l.description
    cityLookup[(longitude, -latitude)] = l.description
    cityLookup[(-longitude, -latitude)] = l.description
    
#make a set with all the landing point coordinates
landPointCoords = set(cityLookup.keys())

In [3]:
#now extract a big set that contains all the coorinates that appear multiple times (important coordinates!)
# Directory with .geojson files
input_dir = "cablesGeojson"
output_file = "all_cables.csv"

def normCoord(long, lat):
    """Converts floating point coordinates to integers for easier comparisons!"""
    return (int(round(long * 1e6)), int(round(lat * 1e6)))

def coordBack(normedCoord):
    """Get the coord back from int to float"""
    return (float(normedCoord[0]) * 1e-6, float(normedCoord[1] * 1e-6))

# Loop through all files in the directory

cableCoords = []
for filename in os.listdir(input_dir):#iterate over all .geojson files
    if filename.endswith(".geojson"):
        filepath = os.path.join(input_dir, filename)
        #print(f"Processing {filepath} ...")

        # Load geojson file
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)

        # Extract cable info
        for feature in data["features"]:
            coords = feature["geometry"].get("coordinates", [])
            if not coords:
                raise ValueError#this should never happen, cables should always have some coordinates!
            coords = [normCoord(c[0], c[1]) for c in coords]#get a list of tuples of coordinates of this cable segment
            cableCoords.extend(coords)#add the coordinates to the list
            
coordSet = set()
duplicateCoordSet = set()#set that will contain all the cable coordinates that appear multiple times
for c in cableCoords:
    c1 = c
    if abs(c[0]) == 180000000:
        c1 = (c[0] * (-1), c[1])#add the negated coordinate because this will allow for connections that cross the 180th latitude
    if c not in coordSet and c1 not in coordSet:#we only consider it "not a duplicate" if neither of the two appear in the current coordinate set
        coordSet.add(c)
    else:
        duplicateCoordSet.add(c)#we found a duplicate
        duplicateCoordSet.add(c1)#add both, if this was a special case at longitude 180

In [4]:

import networkx as nx
import pandas as pd

relevantCoords = duplicateCoordSet.union(landPointCoords)#these will be all the important coordinates!
#now go through all the cables again and for each one add edges between all pairs of relevant coordinates with weight equal to the cable's capacity

G = nx.Graph()
cableCapacities = pd.read_csv("withCapacity.csv")

oneCount = 0#how often was there a cable that wasn't added because only 1 of its coordinates was important?
zeroCount = 0#how often was there a cable that had no relevant coordinate at all???

# Loop through all files in the directory
for filename in os.listdir(input_dir):
    if filename.endswith(".geojson"):
        filepath = os.path.join(input_dir, filename)

        # Load geojson file
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # Extract cable info
        for feature in data["features"]:
            name = feature["properties"].get("Name", "Unknown")
            coords = feature["geometry"].get("coordinates", [])
            if not coords:
                raise ValueError#shouldn't happen
            coords = [normCoord(c[0], c[1]) for c in coords]#get a list of tuples of coordinates of this cable segment
            relevantCs = [c for c in coords if c in relevantCoords]#get only the relevant coordinates from this cable segment

            if len(relevantCs) == 0:#add the edge anyways, both endpoints will be marked as "undersea"
                G.add_edge(coords[0], coords[-1], name=name, capacity=cableCapacities[cableCapacities["name"] == name]["capacity"].values[0])#add the edge with the cable capacity & name
                zeroCount+=1
            if len(relevantCs) == 1:
                G.add_node(relevantCs[0], description="singleton")#WARNING: this name will actually be overwritten by <cityname> if this is a city!
                oneCount+=1
            for i in range(len(relevantCs)-1):#add edge between adjacent important points (adjacent on the cable)
                G.add_edge(relevantCs[i], relevantCs[i+1], name=name, capacity=cableCapacities[cableCapacities["name"] == name]["capacity"].values[0])#add the edge with the cable capacity & name
            # for i in range(len(relevantCs)):
            #     for j in range(i+1, len(relevantCs)):
            #         G.add_edge(relevantCs[i], relevantCs[j], name=name, capacity=cableCapacities[cableCapacities["name"] == name]["capacity"].values[0])#add the edge with the cable capacity & name

print(oneCount)
print(zeroCount)                 
print(G)

58
6
Graph with 1307 nodes and 1531 edges


In [5]:
#now merge the nodes at the 180/-180 longitude
latitudesDone = set()#mark the latitudes that we have already worked on -> since twins are on the same latitudes, this ensures we only work on each pair once!
toDelete = []
for n, d in G.nodes(data=True):
    if abs(n[0]) == 180000000:
        if n[1] not in latitudesDone:#only work on this node if we haven't worked on it's twin already!
            neighborSelf = list(G.neighbors(n))[0]
            neighborTwin = list(G.neighbors((-n[0], n[1])))[0]
            edgeData = list(G.edges(n, data=True))[0][2]#get the data from the only edge of self    ->  assumed to be equal with the data from the other side (they really should be!)
            descr = edgeData["name"]
            cap = edgeData["capacity"]
            G.add_edge(neighborSelf, neighborTwin, name=descr, capacity=cap)#add the new edge
            toDelete.append(n)#add self for deletion
            toDelete.append((-n[0], n[1]))#add the twin for deletion

G.remove_nodes_from(toDelete)

#for some reason there is 1 edge that just get's lost -> add it manually!
anaduir = normCoord(177.50788, 64.73339)
undersea = normCoord(34.934425999999995, 70.038511)
G.add_edge(anaduir, undersea, name="ROTACS", capacity=0.0)

In [6]:
#now rename the stuff

# Build a mapping old_node -> new_node_id
mapping = {}
new_attrs = {}
for idx, node in enumerate(G.nodes()):
    lon, lat = node
    city_name = None
    #now try to match to a city
    if (lon, lat) in cityLookup:
        city_name = cityLookup.get((lon, lat))
    else:
        city_name = "undersea"
    
    mapping[node] = idx  # old coordinate node -> new integer id
    new_attrs[idx] = {
        "coord": (lon, lat),#assign the proper coordinate!
        "description": city_name
    }

# Relabel nodes
G = nx.relabel_nodes(G, mapping)

# Add the attributes
nx.set_node_attributes(G, new_attrs)

#test which cities are not matched:
citiesMatched = []
for n, d in list(G.nodes(data=True)):
    #print(n, d)
    citiesMatched.append(d["description"])
print(len(set(citiesMatched)))

cities = set(list(cityLookup.values()))
print(cities.difference(citiesMatched))

652
{'Boma (Democratic Republic of Congo)', 'Hawaii (USA)', 'Ras Sudar (Egypt)', 'Port Hedland (Australia)', 'Grover Beach (USA)', 'Broadstairs (United Kingdom)', "Cape D'Aguilar (Hong Kong)", 'Cairo (Egypt)', 'Portimao (Portugal)', 'Reykjav�k (Iceland)', 'Wrangell (USA)', 'Sangano Village (Cable Station) (Angola)', 'Castricum (Netherlands)', 'South La (Ghana)', 'Izmir (Turkey)', 'Gambell BMH (USA)', 'Los Angeles (USA)', 'Puerto Viejo (Venezuela)', 'Massawa (Eritrea)', 'Berbera (Somalialand)', 'Punta Gorda (Venezuela)', 'Veules-les-Roses (France)', 'Vale (Guernsey)', 'Skewjack (UK)', 'Seaford (United Kingdom)'}


In [7]:
for e in list(G.edges(data=True))[:5]:
    print(e)

(0, 1, {'name': 'AAE-1 (Asia Africa Europe)', 'capacity': np.float64(0.0)})
(0, 2, {'name': 'AAE-1 (Asia Africa Europe)', 'capacity': np.float64(0.0)})
(0, 3, {'name': 'AAE-1 (Asia Africa Europe)', 'capacity': np.float64(0.0)})
(1, 459, {'name': 'SEA-ME-WE-5', 'capacity': np.float64(0.0)})
(1, 373, {'name': 'SEA-ME-WE-5', 'capacity': np.float64(0.0)})


In [8]:
#remove redundant nodes: (the ones on the seafloor that connect to the same cable twice)
for n, d in list(G.nodes(data=True)):
    if d["description"] == "undersea":
        if G.degree[n] == 2:
            edges = list(G.edges(n, data=True))
            if edges[0][2]["name"] == edges[1][2]["name"]:
                neighbors = list(G.neighbors(n))#there will be 2 neighbors
                G.add_edge(neighbors[0], neighbors[1], name=edges[0][2]["name"], capacity=edges[0][2]["capacity"])#add an edge between the two neighbors with the same cable name & capacity
                G.remove_node(n)
                
print(G)


Graph with 1031 nodes and 1279 edges


In [9]:
#visualize the graph with folium
import folium

def plot_graph_folium(G, output_file="graph_map.html", overlayGeojson = False):
    # Extract coordinates from node attributes
    coords = [coordBack(d["coord"]) for _, d in G.nodes(data=True)]
    lons, lats = zip(*coords)
    
    # Center map on average position
    center_lat = sum(lats) / len(lats)
    center_lon = sum(lons) / len(lons)
    m = folium.Map(location=[center_lat, center_lon], zoom_start=3, tiles="CartoDB positron")

    # Add nodes
    for n, d in G.nodes(data=True):
        lon, lat = coordBack(d["coord"])
        desc = d.get("description", "")
        location = f"lon: {lon} lat: {lat}"
        folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            popup=f"Node {n}<br>{desc}<br>{location}",
            color="blue",
            fill=True,
            fill_color="blue",
            fill_opacity=0.7
        ).add_to(m)

    # Add edges
    for u, v in G.edges():
        lon1, lat1 = coordBack(G.nodes[u]["coord"])
        lon2, lat2 = coordBack(G.nodes[v]["coord"])
        folium.PolyLine(
            locations=[(lat1, lon1), (lat2, lon2)],
            color="red",
            weight=2,
            opacity=0.6
        ).add_to(m)
    
    #overlay the "real network"
    if overlayGeojson:
        # Loop through all files in the directory
        for filename in os.listdir(input_dir):
            if filename.endswith(".geojson"):
                filepath = os.path.join(input_dir, filename)
                folium.GeoJson(filepath).add_to(m)

    # Save map as HTML
    m.save(output_file)
    print(f"Map saved to {output_file}")
    
plot_graph_folium(G, output_file="graph_map_overlayed.html", overlayGeojson=True)
plot_graph_folium(G, output_file="graph_map.html", overlayGeojson=False)

Map saved to graph_map_overlayed.html
Map saved to graph_map.html


In [10]:
#saving the graph to a file:

nx.write_graphml(G, "subsea_cables.graphml")#doesn't work because we have tuple attributes

TypeError: GraphML does not support type <class 'tuple'> as data values.

In [ ]:

#Persisting issues:

#only like 289 out of the 703 landing stations are actually matched! -> that is wayyyy less than what I had yesterday, so something is funky!
#(they should match up exactly even with floats)

#disconnected components are present    ->  this doesn't make sense for the web
#e.g. Copenhagen is only connected to Malmö. Why???

#some cities are straight up missing: Banjul is a good example: the coordinates match up exactly, but it is marked as "undersea" (also Beirut & Tartus)

#a lot of redundant nodes on the sea floor -> they are just intermediate nodes of a single cable (degree 2) -> we can get rid of them,
#perhaps just by filtering for "undersea" nodes with degree 2 and then deleting them (connecting their two neighbors directly while making sure
# that actually both edges belong to the same cable with the same capacity!)
#   -> I fixed it, got rid of about 100 nodes and 100 edges!

#some cables seem to be passing over the land   ->  check if they actually are correct (or if there should be intermediate undersea nodes at
# intersections)

#some nodes have insanely high degree, like New York -> do they actually have that many edges???
#like what the hell is going on between NY and "Sennen Cove" in the UK???

#what the hell is going on with the giant hairball in the carribean sea? and in front of china's coast?



#   some cities are singletons! Now I added them, but they don't have any edges! -> fix it by getting getting the coordinate at the other end of the cable (even though we know it's not a relevant coordinate) and adding it
#   like for example the cable from lon: 34.934425999999995 lat: 70.038511 that goes into nothingness above russia
#   why is the US west coast not connected to China's east coast?


#Sitka USA is a singleton! (somehow the cable from Angloon just doesn't get picked up???)